In [1]:
import pandas as pd
from pathlib import Path
from src.config import lesion_level_labels_csv, dmtr_csv

dmtr = pd.read_csv(dmtr_csv).set_index('id')
dmtr = dmtr[dmtr.center != 'avl']
flow = pd.read_csv('/mnt/c/Users/user/data/tables/patient_flow.csv', sep=';').set_index('id').replace({'yes':True,'no':False,float('nan'):False})

flow = flow.loc[dmtr.index]
flow = flow.join(~dmtr['dcb'].isna())

print('Eligible = ', len(flow))
with_data = flow[flow.data]
print(f'Data available = {len(with_data)} (excluded {len(flow) - len(with_data)})' )
with_ce = with_data[with_data.ce]
print(f'Contrast-enhanced = {len(with_ce)} (excluded {len(with_data) - len(with_ce)})' )
with_lesion = with_ce[with_ce['target lesion']]
print(f'Target lesions = {len(with_lesion)} (excluded {len(with_ce) - len(with_lesion)})' )
with_outcome = with_lesion[with_lesion.dcb]
print(f'Outcome known = {len(with_outcome)} (excluded {len(with_lesion) - len(with_outcome)})' )

included = with_outcome

Eligible =  1347
Data available = 1228 (excluded 119)
Contrast-enhanced = 830 (excluded 398)
Target lesions = 748 (excluded 82)
Outcome known = 728 (excluded 20)


/tmp/ipykernel_1687/2165054813.py:5: DtypeWarning: Columns (1,27,29,175,229,234,249,259,275,281,313,318,351,386,393,440,488,496,538,547,550,579,581,582,583,584,585,587,588,589,599,600,602,603,604,605,610,781,785,786,809,813,814,816,817,820,821,824,906,907,909,910,911,913,916,940,941,943,944,945,952,953,955,956,957,958,959,961,962,963,964,965,967,968,969,970,971,973,974,975,976,989,990,992,993,994,995,996,998,999,1000,1001,1002,1004,1005,1006,1007,1008,1010,1011,1012,1016,1026,1030,1033,1045,1048) have mixed types. Specify dtype option on import or set low_memory=False.
  dmtr = pd.read_csv(dmtr_csv).set_index('id')


In [10]:
from src.data import CENTERS

r = Path('/mnt/d/premium_data/')

dfs = []
for center in CENTERS:
    df = pd.read_csv(r / center / 'ct_parameters.csv')
    dfs.append(df)

df = pd.concat(dfs).set_index('Unnamed: 0')
df = included.join(df).join(dmtr[['center']])

df['Pixel spacing'] = df['Pixel spacing'].apply(lambda x: float(x.split(',')[0][1:]) if type(x) == str else x)

In [12]:
from tableone import TableOne



TableOne(
    df,
    columns=['Current','Voltage','Slice thickness','Pixel spacing','Vendor','Model'],
    groupby='center',
    nonnormal=['Current','Voltage','Slice thickness','Pixel spacing']
).to_csv('/home/rens/repos/premium_dl_ct/tables/scan_parameters.csv')

In [18]:
included

,therapy,data,ce,target lesion,report,dcb
id,,,,,,
PREM_AM_001,Anti-PD1,True,True,True,False,True
PREM_AM_002,Anti-PD1,True,True,True,False,True
PREM_AM_003,Anti-PD1,True,True,True,False,True
PREM_AM_005,Anti-PD1,True,True,True,False,True
PREM_AM_008,Anti-PD1,True,True,True,False,True
...,...,...,...,...,...,...
PREM_ZU_003,Ipilimumab & Nivolumab,True,True,True,True,True
PREM_ZU_002,Anti-PD1,True,True,True,True,True
PREM_ZU_031,Anti-PD1,True,True,True,False,True
